In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import adhtools.utils
import pandas as pd
import os
import numpy as np

## Load corpus

In [19]:
import glob

book_files = glob.glob('/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-13-Fiqh-stemmed-chapters/*.xml')
fnames = [os.path.basename(fn) for fn in book_files]
print(len(book_files))

10066


In [20]:
path_to_stopwordlist = '/media/sf_VBox_Shared/Arabic/arabic_stop-words_7-8-2018.txt'
external_stopwords = [line.strip() for line in open(path_to_stopwordlist, 'r', encoding='utf-8')]

In [21]:
corpus = adhtools.utils.corpus_str(book_files, analyzer=False, field='proposed_root')

## Make document-term matrix

In [22]:
vectorizer = CountVectorizer( stop_words=external_stopwords, min_df=2, max_df=0.9)
X = vectorizer.fit_transform(corpus)

In [23]:
feature_names = vectorizer.get_feature_names()
len(feature_names)

29755

In [24]:
# some example words from the first document
np.array(feature_names)[X[0].indices][:50]

array(['بقل', 'ربط', 'جدي', 'رفق', 'شرط', 'خوص', 'منع', 'قطع', 'بلد',
       'غرر', 'وإنا', 'طوق', 'زخرف', 'قرن', 'ذلل', 'سخر', 'سيأ', 'فوت',
       'حول', 'وسي', 'مدي', 'همز', 'كوف', 'فتح', 'وكآبة', 'همل', 'بسأ',
       'وعث', 'عرف', 'حرف', 'سرد', 'فذلك', 'سبع', 'وسع', 'قعد', 'قيأ',
       'شخص', 'هيأ', 'مضطجع', 'قذر', 'لحم', 'بات', 'أجر', 'عطي', 'فنادق',
       'فهم', 'غرب', 'رخص', 'رغث', 'قمل'], dtype='<U19')

In [53]:
most_imp_words = pd.Series(np.array(feature_names)[X.argmax(axis=1)].flatten())
most_imp_words.value_counts().head(20)

صلي    563
سلم    477
بيع    412
شهد    242
طلق    202
حرم    192
قتل    164
ملك    148
كتب    145
عتق    143
زكي    130
صوم    121
زوج    115
وصي    112
قرر    111
رهن    107
أبن    107
ثلث    105
دعا     99
ولد     98
dtype: int64

## Train model

In [34]:
nr_topics = 10
lda = LatentDirichletAllocation(n_components=nr_topics, random_state=0, max_iter=50)
document_topics = lda.fit_transform(X)

## Save files

In [35]:
fp_out = '/media/sf_VBox_Shared/Arabic/Analyses/topicmodelling/'

In [36]:
df_document_topics = pd.DataFrame(document_topics, index=fnames)
df_document_topics.to_csv(os.path.join(fp_out, 'document_topics_{}.csv'.format(nr_topics)))

In [54]:
lda.components_.shape

(10, 29755)

In [66]:
topic_words = pd.DataFrame(np.argsort(lda.components_, axis=1)[:,-10:][::-1])

In [68]:
topic_words = topic_words.applymap(lambda l: feature_names[l])

In [70]:
topic_words.to_csv(os.path.join(fp_out, 'topics_{}.csv'.format(nr_topics)))

In [69]:
topic_words

,0,1,2,3,4,5,6,7,8,9
0,قطع,قوم,يود,ثبت,حقق,حكم,قضي,قرر,دعا,شهد
1,عبر,علم,ذكر,خبر,خلف,عرف,ظهر,غور,سلم,عدم
2,خلف,غور,طهر,قوم,سلم,أمم,ركع,غسل,سجد,صلي
3,شرط,قوم,عقد,ملك,ثمن,رهن,قبض,جوز,شري,بيع
4,قوم,نصف,ولد,كتب,موت,قسم,وصي,عتق,أبن,ثلث
5,فلي,جور,قوم,زكي,جوز,أخذ,مول,عمل,أجر,ملك
6,ملك,دخل,ولد,مرأ,ثلث,وطأ,فلي,نكح,زوج,طلق
7,غور,وجب,حلل,جزأ,كفر,حوج,يوم,أكل,صوم,حرم
8,وبأ,فلي,صحي,الخ,خلف,شرح,ظهر,ذكر,غور,اه
9,روي,عمر,نبأ,حمد,فى,رسل,أبن,صلي,حدث,سلم
